In [2]:
import pandas as pd
from transformers import AutoProcessor, MllamaForConditionalGeneration
import torch
import os
from dotenv import load_dotenv  


load_dotenv()

hf_token = os.getenv("HF_TOKEN")  

MODEL_ID = "unsloth/Llama-3.2-11B-Vision"
print("Loading model and processor...")


model = MllamaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16, 
    device_map=None,  
    token=hf_token  
)


model.cuda()

processor = AutoProcessor.from_pretrained(MODEL_ID, token=hf_token)

print("Model fully loaded on CUDA.")


Loading model and processor...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model fully loaded on CUDA.


In [6]:
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df)} records from {CSV_PATH}")



predictions = []  
ground_truths = [] 

print("Starting inference on images...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    
    rel_img_path = row["image_path"]
    gt_text = row["ocr_text"]
    
    
    full_img_path = os.path.join(IMAGE_DIR, rel_img_path)
    
    
    try:
        image = Image.open(full_img_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {full_img_path}: {e}")
        continue

Loaded 27 records from /teamspace/studios/this_studio/ocr_results.csv
Starting inference on images...


100%|██████████| 27/27 [00:00<00:00, 69.06it/s]


In [ ]:
import os
from PIL import Image

# Specify the path to your single image
image_path = "/teamspace/studios/this_studio/test/india_news_p000141.jpg"

# Attempt to load the image
try:
    image = Image.open(image_path).convert("RGB")
except Exception as e:
    print(f"Error loading image {image_path}: {e}")
    exit(1)

# Define the prompt for text extraction
prompt = "<|image|><|begin_of_text|>Identify and extract all textual content from this newspaper page while preserving the logical reading flow. Ensure that text spanning across images or boxed sections is correctly ordered. Ignore purely decorative elements and prioritize complete sentences. Maintain proper line breaks and paragraph structure."


# Process the image and prompt using the processor, then move the inputs to the model's device
inputs = processor(image, prompt, return_tensors="pt").to(model.device)

# Generate output from the model with a maximum of 1024 new tokens
output = model.generate(**inputs, max_new_tokens=1024)

# Decode the output and remove any surrounding whitespace
extracted_text = processor.decode(output[0]).strip()

# Print the extracted text
print(f"Extracted text: {extracted_text}")


Extracted text: <|begin_of_text|><|image|><|begin_of_text|>Identify and extract all textual content from this newspaper page while preserving the logical reading flow. Ensure that text spanning across images or boxed sections is correctly ordered. Ignore purely decorative elements and prioritize complete sentences. Maintain proper line breaks and paragraph structure. I'm not able to provide information about the image's purpose. 1. The Patents (Amendment) Act, 1999 passed by the Indian Parliament on March 10, 1999 to amend the Patents Act of 1970 that provides for establishment of a mail box system to file patents and accords exclusive marketing rights for 5 years. 2. The Trade Marks Bill, 1999, which repeals and replaces the Trade and Merchandise Marks Act, 1958 passed by the Indian Parliament in the Winter Session that concluded on December 23, 1999. The Copyright (Amendment) Act, 1999 passed by both houses of the Indian Parliament, and signed by the President of India on December 30

In [7]:
import zipfile
import os

def unzip_file(zip_file_path, extract_to):
    """
    Unzips the given zip file to the specified directory.
    
    Args:
        zip_file_path (str): The path to the zip file.
        extract_to (str): The directory where files will be extracted.
    """
    # Ensure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Open the zip file in read mode
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents into the given directory
        zip_ref.extractall(extract_to)
        print(f"Extracted all files to: {extract_to}")

# Example usage:
if __name__ == "__main__":
    zip_file_path = "/teamspace/studios/this_studio/test.zip"   # Replace with your zip file path
    extract_to = "./"    # Replace with your desired extraction directory
    unzip_file(zip_file_path, extract_to)


Extracted all files to: ./


In [ ]:
predictions = []   
ground_truths = [] 

print("Starting inference on images...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    
    rel_img_path = row["image_path"]
    gt_text = row["ocr_text"]
    
    full_img_path = os.path.join(IMAGE_DIR, rel_img_path)
    
 
    try:
        image = Image.open(full_img_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {full_img_path}: {e}")
        continue

 
    prompt = "<|image|><|begin_of_text|>Identify and extract all textual content from this newspaper page while preserving the logical reading flow. Ensure that text spanning across images or boxed sections is correctly ordered. Ignore purely decorative elements and prioritize complete sentences. Maintain proper line breaks and paragraph structure."



    inputs = processor(image, prompt, return_tensors="pt").to(model.device)


    output = model.generate(**inputs, max_new_tokens=1024)
    

    extracted_text = processor.decode(output[0]).strip()


    predictions.append(extracted_text)
    ground_truths.append(gt_text)
    
    
    print(f"\nImage: {rel_img_path}")
    print(f"Ground Truth: {gt_text}")
    print(f"Extracted:    {extracted_text}")


Starting inference on images...


  4%|▎         | 1/27 [00:40<17:35, 40.59s/it]


Image: /teamspace/studios/this_studio/test/india_news_p000059.jpg
Ground Truth: CLINTON VISIT normalcy has been restored to the bilateral rela- (Continued from page 1) tionship, although some issues still remain to be economic, scientific and technological assistance resolved. President Clinton's forthcoming visit to that, among others, made the Green Revolution India is an opportunity for both countries to in the mid-|960s possible build a new relationship between the world 's two largest democracies in the 2Ist century based The end of the Cold War in the 1990s, coinciding on their common strengths, values and interests. with the  liberalization of the Indian economy, saw a steady improvement in India-US relations Bilateral trade between the two countries has with the Clinton Administration identifying India exceeded US $12 billion in 1999. There was a as one of the 10 major emerging markets. The decline in FDI inflows from the US in 1998 (US last few years have witnessed a number o

  7%|▋         | 2/27 [01:20<16:38, 39.95s/it]


Image: /teamspace/studios/this_studio/test/india_news_p000026.jpg
Ground Truth: 3. Nuclear Forces telligence and information (C412) systems shall be as- 3.1. India's nuclear forces will be effective, enduring, sured: diverse, flexible, and responsive to the requirements in accordance with the concept of credible minimum deter- 5.5. The Indian defence forces shall be in a position to, rence These forces will be based on a triad of aircraft, execute operations in an NBC environment with minimal mobile land-based missiles and sea-based assets in degradation; with the objectives outlined above. 5.6. Space based and other assets shall be created to Survivability of the forces will be enhanced by a combi- provide early warning, communications, damage/deto- nation of multiple redundant systems, mobility, nation assessment: sion and deception. 6. Security and Safety 3.2. The doctrine envisages assured capability to shift from peacetime deployment to fully employable forces in 6.1- Security: E

...


Image: /teamspace/studios/this_studio/test/india_news_p000029.jpg
Ground Truth: Note n Violation of India's airspace Pakistan's Atlantique aircraft and consequent action On August 10, 1999, a Pakistani Naval Anti-Subma- It is crucial to recognise that surveillance recon- rine Warfare and maritime reconnaissance   aircraft, naissance 1 intelligence   gathering is an offensive called Atlantique intruded 10 Kms into Indian Territory military operation and a hostile activity. It cannot be in the Area of KORI CREEK. passed off as harmless: The intruding aircraft was detected by IAF ground ra- The central issue therefore is not whether Pakistan dars and was intercepted 10 Kms south of the Interna- aircraft was armed' Or unarmed" The issue is that tional Border: When the IAF fighters closed in to iden- it was a military aircraft engaged in offensive and tify and signal the intruding Pakistani aircraft to force hostile military operation compounded by its com- it to land at an Indian base, th

 96%|█████████▋| 26/27 [17:08<00:39, 39.46s/it]


Image: /teamspace/studios/this_studio/test/india_news_p000039.jpg
Ground Truth: IIN THIS ISSUE India News is published by the Press & Information Embassy of India. An electronic edition is available at the Embassy'$ web site: Address to the Nation by hllp:Ilwww.indianembassy org Atal Bihari Vajpayee You can also receive an email version of India News. Join te Embassy Announcement List by sending email to indianembassy @ Election Results of 199 egroups com or visit te following site: http:llwww egroups com/grouplindianembassyhnfu html Trade Events _ The Embassy also maintains discussion group for India News readers. You can join by sending email t0: Excerpts from Ambassador Naresh india_discussion @egroups com Chandra's interview on NewsHour or by visiting the following site: on Senates rejection to ratify the http:llwwwegroups cOm/gruuplindia_discussionfinfohtml Nuclear Test Ban Treaty Embassy of India Prime Minister and the Cabinet 5 Press & Information @ 2107 Massachusetts Ave-, NW 

100%|██████████| 27/27 [17:47<00:00, 39.56s/it]


Image: /teamspace/studios/this_studio/test/india_news_p000064.jpg
Ground Truth: Indian American Community: A Story of Achievements There are now more than 1.5 million peoples of Indian origin in America. reflect the multi-ethnic, multi-religious and multi-lingual society of India. Indian-Americans are represented in many fields including academics and entrepreneurs, doctors and lawyers, engineers and financiers: According to the U.S. Census Bureau, Indian-American median family income is 560,093 as against the national median family income of $38, 885. The high income clearly reflects the advanced educational levels achieved by the community: More than 87 % of Indians in America have completed high school while at least 62% have some college education: As much as 58% of Indian Americans over the age of 25 hold a bachelor's degree or higher: High levels of education have also enabled Indian-Americans to become a productive segment of the U. S. population, with 72.3 % participating in t

In [6]:
import pandas as pd


results_df = pd.DataFrame({"image_path": df["image_path"], "ground_truth": ground_truths, "prediction": predictions})


results_csv_path = "predictionsunsloth.csv"
results_df.to_csv(results_csv_path, index=False, encoding="utf-8")

print(f"Predictions saved to {results_csv_path}")


Predictions saved to predictionsunsloth.csv


In [2]:
import pandas as pd


df = pd.read_csv("//teamspace/studios/this_studio/wer_cer.csv")


unwanted_text = "<|begin_of_text|><|image|><|begin_of_text|>Extract the text from this image."
df["prediction"] = df["prediction"].str.replace(rf"^{unwanted_text}\s*", "", regex=True)


df["prediction"] = df["prediction"].str.replace(r"^\|+", "", regex=True).str.lstrip()


df.to_csv("wer_cer.csv", index=False)


In [3]:
import pandas as pd

# Load the two CSV files
csv1_path = "/teamspace/studios/this_studio/cleaned_fileunsloth.csv"  
csv2_path = "/teamspace/studios/this_studio/groud_truth_test.csv"

# Read the CSV files
df1 = pd.read_csv(csv1_path)
df2 = pd.read_csv(csv2_path)

# Copy 'prediction' column from df1 to df2 based on 'image_path'
df2 = df2.merge(df1[['image_path', 'prediction']], on='image_path', how='left')

# Save the updated dataframe to a new CSV file
output_path = "wer_cer_alt.csv"  # Replace with the desired output path
df2.to_csv(output_path, index=False)

print(f"Updated CSV saved to {output_path}")

Updated CSV saved to wer_cer_alt.csv


In [6]:
import pandas as pd
import editdistance
from jiwer import wer


csv_path = "wer_cer_alt.csv"
df = pd.read_csv(csv_path)


if "prediction" not in df.columns or "ground_truth" not in df.columns:
    raise ValueError("CSV file must contain 'prediction' and 'ground_truth' columns.")


predictions = df["prediction"].astype(str).tolist()
ground_truths = df["ground_truth"].astype(str).tolist()

print("\nEvaluating performance...")

total_wer = 0.0
total_cer = 0.0
num_samples = len(predictions)

for pred, gt in zip(predictions, ground_truths):
   
    sample_wer = wer(gt, pred)
    total_wer += sample_wer
    
    
    sample_cer = editdistance.eval(pred, gt) / len(gt) if len(gt) > 0 else 0.0
    total_cer += sample_cer

average_wer = total_wer / num_samples if num_samples > 0 else float('nan')
average_cer = total_cer / num_samples if num_samples > 0 else float('nan')

print(f"\nAverage Word Error Rate (WER): {average_wer:.4f}")
print(f"Average Character Error Rate (CER): {average_cer:.4f}")



Evaluating performance...

Average Word Error Rate (WER): 1.0075
Average Character Error Rate (CER): 0.8006


# The following code finetunes the  llama 3.2 vision model using lora with the train split of the dataset 
## I used unsloth in order to make the training faster

In [ ]:
import cv2
import os

def crop_left_bottom(image_path, left_crop_percent=10, bottom_crop_percent=10):
    """Crop only the left and bottom borders of an image."""
    image = cv2.imread(image_path)

    if image is None:
        raise ValueError(f"Could not open image: {image_path}")

    h, w, _ = image.shape  # Get image dimensions


    crop_x = int(w * left_crop_percent / 100)  
    crop_y = int(h * bottom_crop_percent / 100)  

    # Apply cropping (keep top and right intact)
    cropped_image = image[:h - crop_y, crop_x:w]

    return cropped_image

def process_images_in_directory(directory, left_crop_percent=10, bottom_crop_percent=10):
    """Process all images in the given directory and replace them with cropped versions."""
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Only process images
            image_path = os.path.join(directory, filename)

            try:
                cropped_image = crop_left_bottom(image_path, left_crop_percent, bottom_crop_percent)

                # Overwrite the original image
                cv2.imwrite(image_path, cropped_image)
                print(f"Processed and replaced: {filename}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

# Set your directory and crop percentages
image_directory = "/teamspace/studios/this_studio/train"
left_crop_percent = 7  # Adjust this to crop more/less from the left
bottom_crop_percent = 7  # Adjust this to crop more/less from the bottom

# Process and replace all images in the directory
process_images_in_directory(image_directory, left_crop_percent, bottom_crop_percent)


Processed and replaced: india_news_p000006.jpg
Processed and replaced: india_news_p000007.jpg
Processed and replaced: india_news_p000008.jpg
Processed and replaced: india_news_p000009.jpg
Processed and replaced: india_news_p000010.jpg
Processed and replaced: india_news_p000011.jpg
Processed and replaced: india_news_p000012.jpg
Processed and replaced: india_news_p000013.jpg
Processed and replaced: india_news_p000017.jpg
Processed and replaced: india_news_p000018.jpg
Processed and replaced: india_news_p000020.jpg
Processed and replaced: india_news_p000021.jpg
Processed and replaced: india_news_p000024.jpg
Processed and replaced: india_news_p000025.jpg
Processed and replaced: india_news_p000027.jpg
Processed and replaced: india_news_p000028.jpg
Processed and replaced: india_news_p000030.jpg
Processed and replaced: india_news_p000031.jpg
Processed and replaced: india_news_p000032.jpg
Processed and replaced: india_news_p000033.jpg
Processed and replaced: india_news_p000035.jpg
Processed and

In [3]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)
model.cuda()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Mllama vision patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA L40S. Max memory: 44.527 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, 
    finetune_language_layers   = True, 
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r = 16,           
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3443,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.vision_model.transformer` require gradients


In [5]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset


class OCRDataset(Dataset):
    def __init__(self, csv_path: str, images_dir: str):
        self.data = pd.read_csv(csv_path)
        self.images_dir = images_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.images_dir, row["image_path"])
        image = Image.open(image_path).convert("RGB")
        return {"image": image, "model_output": row["model_output"]}


ocr_dataset = OCRDataset(
    csv_path="/teamspace/studios/this_studio/ground_truth_train.csv",
    images_dir="/teamspace/studios/this_studio/train",
)

dataset = [ocr_dataset[i] for i in range(min(500, len(ocr_dataset)))]

instruction = """
Identify and extract all textual content from this newspaper page while preserving the logical reading flow. Ensure that text spanning across images or boxed sections is correctly ordered. Ignore purely decorative elements and prioritize complete sentences. Maintain proper line breaks and paragraph structure."
"""

def convert_to_conversation(sample):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": sample["image"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["model_output"]}],
        },
    ]
    return {"messages": conversation}


converted_dataset = [convert_to_conversation(sample) for sample in dataset]

print(converted_dataset[45])


{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': '\nIdentify and extract all textual content from this newspaper page while preserving the logical reading flow. Ensure that text spanning across images or boxed sections is correctly ordered. Ignore purely decorative elements and prioritize complete sentences. Maintain proper line breaks and paragraph structure."\n'}, {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=1622x2114 at 0x7F821D46E950>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'India\'s Experience Has Taught That Peace Lies in Strength\n\nBy Atal Bihari Vajpayee, Prime Minister of India\n\nInternational Herald Tribune - September 21, 2000\n\nM Y RECENT visit to the United States has consolidated relations between the world\'s two largest democracies. The joint initiatives and understandings that we reached represent a major step forward. India and the United States can be natural allies in the 21st century. Events and circum

In [6]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) 
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),  
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=1,         
        gradient_accumulation_steps=8,          
        warmup_steps=10,                        
        max_steps=100,                           
        learning_rate=1e-4,                      
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=10,                        
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        dataset_num_proc=4,
        max_seq_length=2048,
    ),
)


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 107 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 67,174,400
🦥 Unsloth needs about 1-3 minutes to load everything - please wait!


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.443700
20,0.155500
30,0.077700
40,0.062700
50,0.055800
60,0.052700
70,0.039700
80,0.037600
90,0.035600
100,0.031900


In [11]:
ocr_dataset = OCRDataset(
    csv_path="/teamspace/studios/this_studio/groud_truth_test.csv",
    images_dir="/teamspace/studios/this_studio/test",
)

dataset = [ocr_dataset[i] for i in range(min(500, len(ocr_dataset)))]
FastVisionModel.for_inference(model)  

image = dataset[9]["image"]

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": instruction},
        ],
    }
]
input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

PRIME MINISTER'S ADDRESS from page 9

Our National Income has grown by 6 percent;
Having crossed 200 million tonnes, production of foodgrains is higher today than ever before;
Food stocks are higher today than ever. For this our kisans deserve our felicitations. Our agricultural scientists are also worthy of our praise.

Industrial production is reviving at an energetic pace;

The new initiatives which have been taken in infrastructure have infused a new purposefulness right across the economy;

At over 30 billion dollars our foreign exchange reserves are higher than ever before;

The Sensex in the stock market has risen to record levels. In spite of Kargil, our companies have been able to increase their market value by over Rs. 200,000 crore.

The off-take of cement for building houses is 22 percent higher than it has ever been;

Facilities which were known only to the rich, to the few, and in our cities alone --- insurance for the output, credit cards --- are now available to, and ar

In [12]:
import pandas as pd
from tqdm import tqdm

# Paths to your CSV and image directory
csv_path = "/teamspace/studios/this_studio/groud_truth_test.csv"
images_dir = "/teamspace/studios/this_studio/test"

# Load the CSV into a DataFrame (this assumes the CSV already contains image paths or identifiers)
df = pd.read_csv(csv_path)

# Create your OCRDataset instance
ocr_dataset = OCRDataset(csv_path=csv_path, images_dir=images_dir)

# Prepare your model for inference (make sure 'model' is already loaded and 'FastVisionModel' is imported)
FastVisionModel.for_inference(model)

# List to collect predictions for each image
predictions = []

# Loop over the entire dataset (one sample per image)
for idx in tqdm(range(len(ocr_dataset)), desc="Processing images"):
    # Retrieve the sample; assumes each sample is a dictionary with key "image"
    sample = ocr_dataset[idx]
    image = sample["image"]
    
    # Prepare the prompt/message.
    # Ensure that the variable 'instruction' is defined with your desired text.
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": instruction},
            ],
        }
    ]
    
    # Apply the chat template to create the input text for the model
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    
    # Tokenize both the image and text input
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")
    
    # Generate the prediction
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        use_cache=True,
        temperature=1.5,
        min_p=0.1
    )
    
    # Decode the generated tokens into text (skip special tokens)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Append the prediction to the list
    predictions.append(prediction)

# Add the predictions as a new column in the DataFrame.
# Make sure the order of 'predictions' matches the rows in your CSV.
df['prediction'] = predictions

# Save the updated DataFrame back to CSV
df.to_csv(csv_path, index=False)


Processing images: 100%|██████████| 27/27 [33:05<00:00, 73.55s/it]


In [15]:
import pandas as pd

# Load the CSV
file_path = "/teamspace/studios/this_studio/groud_truth_test.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Define the exact multiline string to remove
text_to_remove = """user


Identify and extract all textual content from this newspaper page while preserving the logical reading flow. Ensure that text spanning across images or boxed sections is correctly ordered. Ignore purely decorative elements and prioritize complete sentences. Maintain proper line breaks and paragraph structure."""

# Remove occurrences of the specific string in the 'prediction' column
df["prediction"] = df["prediction"].str.replace(text_to_remove, "", regex=False)

# Save the cleaned CSV
df.to_csv("cleaned_file_final.csv", index=False)


In [11]:
import os
import pandas as pd
from PIL import Image
import torch
from tqdm import tqdm
from jiwer import wer
import editdistance

from transformers import MllamaForConditionalGeneration, AutoProcessor


CSV_PATH = "/teamspace/studios/this_studio/groud_truth_test.csv"      
IMAGE_DIR = "/teamspace/studios/this_studio/test"       
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["ocr_text"])
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df)} records from {CSV_PATH}")



predictions = []  
ground_truths = [] 

print("Starting inference on images...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    
    rel_img_path = row["image_path"]
    gt_text = row["ocr_text"]
    
    
    full_img_path = os.path.join(IMAGE_DIR, rel_img_path)
    
    
    try:
        image = Image.open(full_img_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {full_img_path}: {e}")
        continue

Loaded 27 records from /teamspace/studios/this_studio/groud_truth_test.csv
Starting inference on images...


100%|██████████| 27/27 [00:00<00:00, 89.09it/s]


In [12]:
model.save_pretrained("llama_3.2_vision_finetuned") 
tokenizer.save_pretrained("llama_3.2_vision_finetuned")

[]

In [14]:
import pandas as pd

# Paths for the original and new CSV files
CSV_PATH = "/teamspace/studios/this_studio/cleaned_file_final.csv"
NEW_CSV_PATH = "/teamspace/studios/this_studio/updated_cleaned_file_final.csv"

# Read the original CSV file
df = pd.read_csv(CSV_PATH)

# Remove occurrences where 'assistant' appears as a standalone line along with extra whitespace/newlines
df["prediction"] = df["prediction"].str.replace(r"\nassistant\s*\n", "\n", regex=True)

# Remove a starting double quote at the beginning of the text or following a newline
df["prediction"] = df["prediction"].str.replace(r'(^|\n)"', r'\1', regex=True)

# Display the first few predictions to verify changes
print(df["prediction"].head())

# Save the updated dataframe to a new CSV file
df.to_csv(NEW_CSV_PATH, index=False)
print(f"Updated CSV saved to {NEW_CSV_PATH}")


0    \nNEWS IN BRIEF (Continued from page 8)\n\nDur...
1    \nIndia not to engage in a nuclear arms race: ...
2    \nIndia News is published by the Press & Infor...
3    \nIndia & Intellectual Property Rights\n\nTHER...
4    \nIndia Press Monitor\n\nWired to the World\nL...
Name: prediction, dtype: object
Updated CSV saved to /teamspace/studios/this_studio/updated_cleaned_file_final.csv


In [15]:
import pandas as pd
import editdistance
from jiwer import wer


csv_path = "/teamspace/studios/this_studio/updated_cleaned_file_final.csv"
df = pd.read_csv(csv_path)

if "prediction" not in df.columns or "ground_truth" not in df.columns:
    raise ValueError("CSV file must contain 'prediction' and 'ground_truth' columns.")


predictions = df["prediction"].astype(str).tolist()
ground_truths = df["ground_truth"].astype(str).tolist()

print("\nEvaluating performance...")

total_wer = 0.0
total_cer = 0.0
num_samples = len(predictions)

for pred, gt in zip(predictions, ground_truths):
  
    sample_wer = wer(gt, pred)
    total_wer += sample_wer
    
 
    sample_cer = editdistance.eval(pred, gt) / len(gt) if len(gt) > 0 else 0.0
    total_cer += sample_cer

average_wer = total_wer / num_samples if num_samples > 0 else float('nan')
average_cer = total_cer / num_samples if num_samples > 0 else float('nan')

print(f"\nAverage Word Error Rate (WER): {average_wer:.4f}")
print(f"Average Character Error Rate (CER): {average_cer:.4f}")



Evaluating performance...



Average Word Error Rate (WER): 0.1041
Average Character Error Rate (CER): 0.0942


## Following code compares the code before and after fine tuning 

In [18]:
import pandas as pd
import editdistance
from jiwer import wer
from nltk.translate.bleu_score import sentence_bleu
from tabulate import tabulate


normal_csv = "/teamspace/studios/this_studio/cleaned_fileunsloth.csv"
finetuned_csv = "/teamspace/studios/this_studio/updated_cleaned_file_final.csv"

df_normal = pd.read_csv(normal_csv)
df_finetuned = pd.read_csv(finetuned_csv)


for df in [df_normal, df_finetuned]:
    if "prediction" not in df.columns or "ground_truth" not in df.columns:
        raise ValueError("CSV files must contain 'prediction' and 'ground_truth' columns.")

def calculate_metrics(predictions, ground_truths):
    total_wer, total_cer, total_edit_distance, total_bleu, exact_matches = 0, 0, 0, 0, 0
    num_samples = len(predictions)
    
    for pred, gt in zip(predictions, ground_truths):
        total_wer += wer(gt, pred)
        total_cer += editdistance.eval(pred, gt) / len(gt) if len(gt) > 0 else 0.0
        total_edit_distance += editdistance.eval(pred, gt)
        total_bleu += sentence_bleu([gt.split()], pred.split()) 
        exact_matches += 1 if pred == gt else 0
    
    return {
        "WER": total_wer / num_samples if num_samples > 0 else float('nan'),
        "CER": total_cer / num_samples if num_samples > 0 else float('nan'),
        "Edit Distance": total_edit_distance / num_samples if num_samples > 0 else float('nan'),
        "BLEU Score": total_bleu / num_samples if num_samples > 0 else float('nan'),
    }


normal_metrics = calculate_metrics(df_normal["prediction"].astype(str).tolist(), df_normal["ground_truth"].astype(str).tolist())
finetuned_metrics = calculate_metrics(df_finetuned["prediction"].astype(str).tolist(), df_finetuned["ground_truth"].astype(str).tolist())


data = [[metric, f"{normal_metrics[metric]:.4f}", f"{finetuned_metrics[metric]:.4f}"] for metric in normal_metrics.keys()]

print("\nPerformance Comparison:")
print(tabulate(data, headers=["Metric", "Normal Model", "Fine-Tuned Model"], tablefmt="grid"))


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain


Performance Comparison:
+---------------+----------------+--------------------+
| Metric        |   Normal Model |   Fine-Tuned Model |
+===============+================+====================+
| WER           |         1.1465 |             0.1041 |
+---------------+----------------+--------------------+
| CER           |         0.9231 |             0.0942 |
+---------------+----------------+--------------------+
| Edit Distance |      3325.07   |           267.926  |
+---------------+----------------+--------------------+
| BLEU Score    |         0.3453 |             0.8848 |
+---------------+----------------+--------------------+
